In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
import warnings
warnings.filterwarnings('ignore')

In [ ]:
df = pd.read_csv('../input/bank-marketing-dataset/bank.csv')

In [ ]:
df.shape 

In [ ]:
df.head() #lets see the data 

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
#bal is in -ve and +ve including 0
#duration and previous has zero values but  completly makes sense and cannot consider as missing values
# campaign atleast once the customer was already contacted during this campaing year 
#min age is 18 year eliglible for opting the term plan
#day will not make sense as it implies which day cust was last contacted

In [ ]:
#null values
df.isnull().sum()

In [ ]:
df.columns

In [ ]:
#rename the col to more meaningful 
df.columns = ['age', 'job', 'marital', 'education', 'default_credit', 'balance', 'house_loan',
       'personal_loan', 'contact_type', 'day_ofweek', 'month', 'duration', 'current_campaign_contact_count', 'days_passed',
       "previous_coutact_count",'previous_campaing_outcome','deposit']

In [ ]:
df.head()

In [ ]:
#lets see the cat features
df_cat = df.select_dtypes(include='O')

In [ ]:
df_cat.head()

In [ ]:
#lets count the unique values in each col

df_cat.nunique()

In [ ]:
#univariate analysis

In [ ]:
#lets see each cat feature 

pos =1
plt.figure(figsize=(100,100))

for i in df_cat.columns:
    sns.set(font_scale = 8)
    plt.subplot(5,2,pos)
    sns.countplot(df[i])
    plt.title(i,fontweight="bold",fontsize=50) 
    pos +=1
    plt.tight_layout()

In [ ]:
df['job'].unique()
label =['management', 'technician', 'entrepreneur', 'blue-collar',
       'unknown', 'retired', 'admin.', 'services', 'self-employed',
       'unemployed', 'housemaid', 'student']

In [ ]:
#5k opted for term deposit
# more cust were contact in may from last contact month of year 
#mostly contacted by calls
#very less customer with default credit 



In [ ]:
sns.set(font_scale = 1)
plt.pie(df['job'].value_counts(),autopct='%.2f',radius=1.2,explode=[0,0,0,0,0.5,0.5,0,0,0,1,0,1],shadow=True,labels=label)

In [ ]:
#~18% of customer exploded in above are student and others not working currently

In [ ]:
df['deposit'].value_counts()

In [ ]:
df[df['deposit']=='yes'].shape[0]*100/df['deposit'].shape[0]
#12 % customer has opted for the term deposit in current campain

In [ ]:
# lets analyse continous features
df_num = df.select_dtypes(exclude='O')

In [ ]:
pos =1
plt.figure(figsize=(100,100))

for i in df_num.columns:
    sns.set(font_scale = 8)
    plt.subplot(5,2,pos)
    sns.distplot(df[i], kde_kws={"color": "k", "lw": 3, "label": "KDE"})
    plt.title(i,fontweight="bold",fontsize=50) 
    pos +=1
    plt.tight_layout()

In [ ]:
#atleast 5 times custoemr were contacted in current_campaign_contact_count
#atmax 25 times custoemr were contacted in prevoius_campaign
#age 30-40 has more cust 
#Durartion and days passed are near to normal distrubution
sns.set(font_scale = 1)
plt.hist(df['age'])
plt.title('duration',fontsize=10)

In [ ]:
sns.set(font_scale = 1)
plt.hist(df['duration'])
plt.title('duration',fontsize=10)
#duration is mostly to make next call is under 500days

In [ ]:
sns.set(font_scale = 1)
plt.hist(df['days_passed'])
plt.title('days_passed',fontsize=10)
#50-100 days passed after last call made from a previous campaign

In [ ]:
pd.crosstab(df['month'],df['deposit']).apply(lambda x:x/x.sum()*100)

In [ ]:

nodep = pd.crosstab(df['deposit'],df['month']).iloc[0]
dep = pd.crosstab(df['deposit'],df['month']).iloc[1]
tot = nodep+dep
print(tot.index)
labels = ['apr', 'aug', 'dec', 'feb', 'jan', 'jul', 'jun', 'mar', 'may', 'nov','oct', 'sep']
plt.rcdefaults()
plt.pie(tot,labels=labels,autopct='%.1f',radius=1.1,)
plt.title('No of targted cust/month')

In [ ]:
sns.scatterplot(dep,labels)
sns.scatterplot(nodep,labels)
plt.title('Customer response after calls')
legend=['no of cust opted montly','no of cust not opted montly']
plt.legend(legend)

In [ ]:
#effectiveness
effc= pd.crosstab(df['month'],df['deposit'])
effc['diff'] = effc['yes']-effc['no']
effc['tot'] = effc['no']+effc['yes']
effc['effective'] = round(effc['diff']/effc['tot'] *100,2)
effc


In [ ]:
!pip install plotly

In [ ]:

import plotly.graph_objs as go
from plotly.offline import iplot, init_notebook_mode
# Add data

ratio = effc['effective']

line = go.Scatter(
    x = effc.index,
    y = effc['effective'],
    text = '(%)',
    fill = 'tonexty',
    mode = 'lines',
    name='Effectiveness Ratio',
    line = dict(
        color = ('#ff3300'),
        width= 4,
        dash = 'dot',
    
    )

)

data = [line]

layout = dict(title='Effectiveness per Month',
             xaxis=dict(title='Month'),
              yaxis=dict(title='Effectiveness ratio')
             )

fig = dict(data=data, layout=layout)
iplot(fig)

In [ ]:
plt.rcdefaults()
data = df.copy()
data['deposit'].replace({'yes':1,'no':0},inplace=True)
data['month'].replace({'jan':1, 'feb':2, 'mar':3, 'apr':4, 'may':5, 'jun':6, 'jul':7, 'aug':8, 'sep':9, 'oct':10, 'nov':11, 'dec':12},inplace=True)
sns.kdeplot(data[data['deposit']== 0]['month'],color='r',label='not opted',shade=True)
sns.kdeplot(data[data['deposit']== 1]['month'],color='g',label='opted',shade=True)
plt.legend()

In [ ]:
#Bivariate analysis
pos =1
plt.figure(figsize=(100,100))

for i in df_cat.columns:
    sns.set(font_scale = 8)
    plt.subplot(5,2,pos)
    sns.countplot(df[i],hue=df['deposit'])
    plt.title(i,fontweight="bold",fontsize=50)
    pos +=1
    plt.tight_layout()

In [ ]:
plt.rcdefaults()
plt.figure(figsize=(50,50))
sns.jointplot(df['duration']/60,df['current_campaign_contact_count'],data=df,hue=df['deposit'])

#AFter 10 call the customers are lossing interest in opting the plan

In [ ]:
df.head()
pd.crosstab(df['deposit'], df['current_campaign_contact_count']).apply(lambda x: x/x.sum() * 100).iloc[:,:5]
#campaiGn wise sucessrate

In [ ]:
plt.rcdefaults()
data = df.copy()
data['deposit'].replace({'no':0,'yes':1},inplace=True)
sns.kdeplot(data[data['deposit']==1]['age'])
sns.kdeplot(data[data['deposit']==0]['age'])

label = ['Opted','NotOpted']
plt.legend(label)

In [ ]:
lst = [df]
for column in lst:
    column.loc[column["age"] < 30,  "age_cat"] = 20
    column.loc[(column["age"] >= 30) & (column["age"] <= 39), "age_cat"] = 30
    column.loc[(column["age"] >= 40) & (column["age"] <= 49), "age_cat"] = 40
    column.loc[(column["age"] >= 50) & (column["age"] <= 59), "age_cat"] = 50
    column.loc[column["age"] >= 60, "age_cat"] = 60
df['age_cat'] = df['age_cat'].astype(np.int64)

In [ ]:
#df.drop('age_cat',axis=1,inplace=True)
df.head()

In [ ]:
plt.rcdefaults()
sns.countplot(df['age_cat'])

In [ ]:
plt.rcdefaults()
sns.countplot(df['age_cat'],hue=df['deposit'])

In [ ]:
pd.crosstab(df['deposit'],df['age_cat'],).apply(lambda x:x/x.sum()*100)

In [ ]:
pd.crosstab(df['deposit'],df['job'],).apply(lambda x:x/x.sum()*100)

In [ ]:
#Pre-Processing
#lets try to handle the Unknown cases

In [ ]:
for i in df_cat:
    print(i,'--->',df[i].unique())
    print('*'*50)

In [ ]:
# we have unknown values in previous_campaing_outcome,job,education

In [ ]:
df.groupby('education')['job'].agg([pd.Series.mode])

In [ ]:
#lets fill the unkown cases in job  with mode of the education

In [ ]:
#lets fill the unkown cases in job  with mode of the education
def Impute(df):
    job = df[0]
    edu = df[1]
    
    if job == 'unknown':
        if edu == 'tertiary': return 'management'
        else: return 'blue-collar'
    else:
        return job

# job Impute
df['job'] = df[['job' , 'education']].apply(Impute, axis = 1)

In [ ]:
df.groupby('job')['education'].agg([pd.Series.mode])

In [ ]:
#lets fill the unkown cases in job  with mode of the education
def Impute(df):
    job = df[1]
    edu = df[0]
    
    if edu == 'unknown':
        if job == 'housemaid': return 'primary'
        elif job == 'entrepreneur' or 'self-employed' or 'management': return 'tertiary'
        else: return 'secondary'
    else:
        return edu

# job Impute
df['education'] = df[[ 'education','job' ]].apply(Impute, axis = 1)

In [ ]:
df['previous_campaing_outcome'].value_counts()

In [ ]:
df.groupby('deposit')['previous_campaing_outcome'].count()

In [ ]:
sns.reset_defaults()
sns.catplot(df['days_passed'],df['previous_campaing_outcome'],data=df,kind='strip')

In [ ]:
# for unknown cases days passed is mostly same value ...lets see what is it

In [ ]:
pd.crosstab(index=df['previous_campaing_outcome'],columns=df['days_passed'])

In [ ]:
#so the value is '-1' which means that previously customer was not contacted 
#lets mark these values as nonexistent

In [ ]:
df['previous_campaing_outcome'].replace('unknown','nonexistent',inplace=True)

In [ ]:
df['previous_campaing_outcome'].value_counts()

In [ ]:
df.drop('age_cat',axis=1,inplace=True)
df.head()

In [ ]:
df['contact_type'].value_counts()

In [ ]:
#Lets impute unknown as non-call medium of contact
df['contact_type'].replace('unknown','non-call',inplace=True)

In [ ]:
#correlation

In [ ]:
corr = df.corr()

In [ ]:
sns.reset_defaults()
sns.heatmap(corr[(corr>0.3)|(corr<-0.3)],vmax=1,vmin=-1,cmap='Blues',annot=True)
plt.title

In [ ]:
#lets see how the correlation is between days_passed and prevoiusly contacted

In [ ]:
plt.rcdefaults()
sns.jointplot(df['days_passed'],df['previous_coutact_count'],hue=df['deposit'])
#not a linear but somepositive correlation exits

In [ ]:
#outliers

In [ ]:
pos =1
plt.figure(figsize=(100,100))

for i in df_num.columns:
    sns.set(font_scale = 8)
    plt.subplot(5,2,pos)
    sns.boxplot(y = df[i],)
    plt.title(i,fontweight="bold",fontsize=50) 
    pos +=1
    plt.tight_layout()

In [ ]:
#% outliers
data = df_num.copy()
for k, v in data.items():
        q1 = v.quantile(0.25)
        q3 = v.quantile(0.75)
        irq = q3 - q1
        v_col = v[(v <= q1 - 1.5 * irq) | (v >= q3 + 1.5 * irq)]
        perc = np.shape(v_col)[0] * 100.0 / np.shape(data)[0]
        print("Column %s outliers = %.2f%%" % (k, perc))

In [ ]:
#days_passed and prevoiusly contacted shows 100% outliers coz of 0 value and -1 value
#seems to be mathamtically outluers

In [ ]:
#lets see thhe outliers in age,bal,contact_out

In [ ]:
from collections import Counter
def detect_outliers(df,features):
    outlier_indices = []
    
    for c in features:
        #1st quartile
        Q1 = np.percentile(df[c],25)
        #3rd quartile
        Q3 = np.percentile(df[c],75)
        #IQR
        IQR = Q3-Q1
        #Outlier Step
        outlier_step= IQR * 1.5
        #Detect outlier and their indices
        outlier_list_col = df[(df[c]<Q1 - outlier_step) | (df[c]> Q3 + outlier_step)].index
        #store indices
        outlier_indices.extend(outlier_list_col)
        
    outlier_indices = Counter(outlier_indices)
    multiple_outliers = list(i for i,v in outlier_indices.items() if v > 1)
    
    return multiple_outliers
df.loc[detect_outliers(df,["age","balance","duration"])]['age'].min()

In [ ]:
#sns.boxplot(y = df['balance'])
Q1 = np.percentile(df['age'],25)
        #3rd quartile
Q3 = np.percentile(df['age'],75)
        #IQR
IQR = Q3-Q1
IQR

In [ ]:
a = Q3+1.5*IQR
b = Q1-1.5*IQR
a,b

In [ ]:
#there are no customner in lower outlier level for age and the customer age above 70.5 are not actually outliers as many of them 
# have opted the term plan so we cannot remove them.


In [ ]:
#So in age makes outlier makes sense with lowwer range 

In [ ]:
#sns.boxplot(y = df['balance'])
Q1 = np.percentile(df['balance'],25)
        #3rd quartile
Q3 = np.percentile(df['balance'],75)
        #IQR
IQR = Q3-Q1
IQR

In [ ]:
a = Q3+1.5*IQR
b = Q1-1.5*IQR
a,b

In [ ]:
#balance cannot be consider here as it has -ve values 0 and +ve , so this seems to be debit credit 
#we can drop the balance column 

In [ ]:
df.drop('balance',axis=1,inplace=True)

In [ ]:
#sns.boxplot(y = df['balance'])
Q1 = np.percentile(df['duration'],25)
        #3rd quartile
Q3 = np.percentile(df['duration'],75)
        #IQR
IQR = Q3-Q1
IQR

In [ ]:
a = Q3+1.5*IQR
b = Q1-1.5*IQR
a,b

In [ ]:
from collections import Counter
def detect_outliers(df,features):
    outlier_indices = []
    
    for c in features:
        #1st quartile
        Q1 = np.percentile(df[c],25)
        #3rd quartile
        Q3 = np.percentile(df[c],75)
        #IQR
        IQR = Q3-Q1
        #Outlier Step
        outlier_step= IQR * 1.5
        #Detect outlier and their indices
        outlier_list_col = df[(df[c]<Q1 - outlier_step) | (df[c]> Q3 + outlier_step)].index
        #store indices
        outlier_indices.extend(outlier_list_col)
        
    outlier_indices = Counter(outlier_indices)
    multiple_outliers = list(i for i,v in outlier_indices.items() if v > 0)
    
    return multiple_outliers
df.loc[detect_outliers(df,["duration"])][['duration','deposit']]

In [ ]:
df.loc[detect_outliers(df,["duration"])][['duration','deposit']].max()

In [ ]:
# similar to age in duration there are no actual outliers even though the duration in seconds is high customer have opted the term plan

In [ ]:
#multivarite analysis

In [ ]:
plt.rcdefaults()
sns.pairplot(df)

In [ ]:
#no linear relation can be seen between features

In [ ]:
#lets encode the catogerical featires

In [ ]:
df.head()

In [ ]:
#All customer are equally important so for banks perpespitive so i label enocde

In [ ]:
df = df.join(pd.get_dummies(df['job'],drop_first=True))
df = df.join(pd.get_dummies(df['marital'],drop_first=True))
df = df.join(pd.get_dummies(df['education'],drop_first=True))
df = df.join(pd.get_dummies(df['contact_type'],drop_first=True))
df = df.join(pd.get_dummies(df['month'],drop_first=True))
df = df.join(pd.get_dummies(df['previous_campaing_outcome'],drop_first=True))



In [ ]:
df.replace({'yes':1,'no':0},inplace=True)


In [ ]:
df.head(1)

In [ ]:

df_cat.columns

In [ ]:
df.drop(['job', 'marital', 'education','contact_type', 'month', 'previous_campaing_outcome'],axis=1,inplace=True)

In [ ]:
pd.set_option('display.max_columns',40)
df.head()

In [ ]:
df = df.drop_duplicates()

In [ ]:
x = df.drop('deposit',axis=1)

In [ ]:
y = df['deposit']

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
xtrain,xtest,ytrain,ytest = train_test_split(x,y,test_size=0.3,random_state=1)

In [ ]:
#lets scale using min_max scaler

In [ ]:
from sklearn.preprocessing import MinMaxScaler

In [ ]:
scl = MinMaxScaler()

In [ ]:
scl.fit(xtrain)

In [ ]:
xtrain = pd.DataFrame(scl.transform(xtrain),columns=x.columns)

In [ ]:
xtest= pd.DataFrame(scl.transform(xtest),columns=x.columns)

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
lr = LogisticRegression()

In [ ]:
lr.fit(xtrain,ytrain)

In [ ]:
pred = lr.predict(xtest)

In [ ]:
lr.score(xtrain,ytrain)

In [ ]:
lr.score(xtest,ytest)

In [ ]:
from sklearn.metrics import confusion_matrix,accuracy_score,roc_auc_score,roc_curve

In [ ]:
cm = confusion_matrix(ytest,pred)
tn, fp, fn, tp = confusion_matrix(ytest,pred).ravel()
lr_recall = round(tp/(tp+fn), 3)   # Of all the people, how many were recognised to opt
lr_precision = round(tp/(tp+fp), 3)  # Of all the people predicted to have opt, how many did have opted
lr_specificity =round(tn/(tn+fp),3)    # Of all the people not opted, how many were recognised to not opted
type_1_error = fp    # People without Chd predicted to have Chd
type_2_error = fn    # People with Chd predicted to have no Chd
lr_recall, lr_precision, lr_specificity, type_1_error, type_2_error

#specificity is good but recall is low


In [ ]:
sns.heatmap(cm,annot=True,fmt='.6g',cmap='Blues',)

In [ ]:
probs = lr.predict_proba(xtest)
# keep probabilities for the positive outcome only
probs = probs[:, 1]
# calculate AUC
auc = roc_auc_score(ytest, probs)
print('AUC: %.3f' % auc)
# calculate roc curve
fpr, tpr, thresholds = roc_curve(ytest, probs)
# plot no skill
plt.plot([0, 1], [0, 1], linestyle='--')
# plot the roc curve for the model
plt.plot(fpr, tpr, marker='.')
# show the plot
plt.show()

In [ ]:
from sklearn.tree import DecisionTreeClassifier

In [ ]:
dt = DecisionTreeClassifier(random_state=1)

In [ ]:
dt.fit(xtrain,ytrain,)

In [ ]:
pred_dt = dt.predict(xtest)

In [ ]:
dt.score(xtrain,ytrain)

In [ ]:
dt.score(xtest,ytest)

In [ ]:
#overfitting lets cv


In [ ]:
from sklearn.model_selection import cross_val_score
tree_clf = DecisionTreeClassifier()
tree_scores = cross_val_score(tree_clf, xtrain, ytrain, cv=10)
tree_mean = tree_scores.mean()
tree_mean

In [ ]:
from sklearn.model_selection import RepeatedStratifiedKFold,GridSearchCV
param_dist = {"max_depth": [1,2,3,5,4,6,7,8,9,None],
              "max_features":[1,2,3,5,4,6,7,8,9,None],
              "min_samples_leaf": [1,2,3,5,4,6,7,8,9,None],
              "criterion": ["gini", "entropy"]}
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
grid_search = GridSearchCV(estimator=DecisionTreeClassifier(), param_grid=param_dist, n_jobs=-1, cv=10, scoring='accuracy',error_score=0)
grid_result = grid_search.fit(x, y)

In [ ]:
grid_result.best_params_

In [ ]:
dt_prun = DecisionTreeClassifier(criterion = 'gini', max_depth =8,min_samples_leaf = 9)

In [ ]:
dt_prun.fit(xtrain,ytrain)

In [ ]:
pred_prun =  dt.predict(xtest)

In [ ]:
dt_prun.score(xtrain,ytrain) 

In [ ]:
dt_prun.score(xtest,ytest) #Overfitting is reduced

In [ ]:
cm = confusion_matrix(ytest,pred_prun)
tn, fp, fn, tp = confusion_matrix(ytest,pred_prun).ravel()
lr_recall = round(tp/(tp+fn), 3)   # Of all the people, how many were recognised to opt
lr_precision = round(tp/(tp+fp), 3)  # Of all the people predicted to have opt, how many did have opted
lr_specificity =round(tn/(tn+fp),3)    # Of all the people not opted, how many were recognised to not opted
type_1_error = fp    # People without Chd predicted to have Chd
type_2_error = fn    # People with Chd predicted to have no Chd
lr_recall, lr_precision, lr_specificity, type_1_error, type_2_error

In [ ]:
sns.heatmap(cm,annot=True,fmt='.6g',cmap='Blues',)

In [ ]:
#We are missclassifying 788 clients who will potentially opt for the term plan

In [ ]:
probs = dt_prun.predict_proba(xtest)
# keep probabilities for the positive outcome only
probs = probs[:, 1]
# calculate AUC
auc = roc_auc_score(ytest, probs)
print('AUC: %.3f' % auc)
# calculate roc curve
fpr, tpr, thresholds = roc_curve(ytest, probs)
# plot no skill
plt.plot([0, 1], [0, 1], linestyle='--')
# plot the roc curve for the model
plt.plot(fpr, tpr, marker='.')
# show the plot
plt.show()

In [ ]:


f1_score = 2*(lr_recall*lr_precision)/(lr_recall+lr_precision)
f1_score

In [ ]:
#lets see the best feature
score=dt_prun.feature_importances_
pd.DataFrame(score,index=x.columns,columns=['score']).sort_values('score',ascending=False).head(10).plot.barh()
